<a href="https://colab.research.google.com/github/BOLLOJUAISHWARYA/Google-collab-ANN/blob/main/Classificationmodel_with_normalization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

LOAN PREDICTION 

Problem statement: Need to predict if the given person will get loan or not based on the given variables

Inference : Binary classification problem


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

Connecting to google drive to import data inserted the code snippet 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Reading the csv file using pandas 


In [ ]:
traindata=pd.read_csv('/content/drive/MyDrive/Datasets/loan predictor/train/train.csv')

In [ ]:
traindata.isnull().sum()

Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [ ]:
# Dropping loan_id
traindata.drop('Loan_ID', axis=1 , inplace=True)

In [ ]:
print(traindata['Credit_History'].unique())
print(traindata['Dependents'].unique())
print(traindata['Property_Area'].unique())


[ 1.  0. nan]
['0' '1' '2' '3+' nan]
['Urban' 'Rural' 'Semiurban']


'Gender', 'Married','Education', 'Self_Employed', 'Property_Area'--categorical

for categorical missing values replaced with mode


In [ ]:
traindata['Gender'].fillna(traindata['Gender'].mode()[0],inplace=True)
traindata['Married'].fillna(traindata['Married'].mode()[0],inplace=True)
traindata['Education'].fillna(traindata['Education'].mode()[0],inplace=True)
traindata['Self_Employed'].fillna(traindata['Self_Employed'].mode()[0],inplace=True)
traindata['Property_Area'].fillna(traindata['Property_Area'].mode()[0],inplace=True)
traindata['Dependents'].fillna(traindata['Dependents'].mode()[0],inplace=True)

In [ ]:
traindata.isnull().sum()

Gender                0
Married               0
Dependents            0
Education             0
Self_Employed         0
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

For numerical missing values replaced with mean

In [ ]:
traindata['LoanAmount'].fillna(traindata['LoanAmount'].mean(),inplace=True)
traindata['Loan_Amount_Term'].fillna(traindata['Loan_Amount_Term'].mean(),inplace=True)
traindata['Credit_History'].fillna(traindata['Credit_History'].mean(),inplace=True)

In [ ]:
traindata.isnull().sum()

Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

Manual encoding

In [ ]:
traindata['Gender'].replace({'Male':1,'Female':0},inplace=True)
traindata['Education'].replace({'Graduate':1,'Not Graduate':0},inplace=True)
traindata['Property_Area'].replace({'Rural':0,'Semiurban':1,'Urban':2},inplace=True)
traindata['Self_Employed'].replace({'Yes':1,'No':0},inplace=True)
traindata['Married'].replace({'Yes':1,'No':0},inplace=True)
traindata['Dependents'].replace({'3+':3},inplace=True)

In [ ]:
traindata.dtypes

Gender                 int64
Married                int64
Dependents            object
Education              int64
Self_Employed          int64
ApplicantIncome        int64
CoapplicantIncome    float64
LoanAmount           float64
Loan_Amount_Term     float64
Credit_History       float64
Property_Area          int64
Loan_Status           object
dtype: object

In [ ]:
traindata['Loan_Status'].replace({'Y':1,'N':0},inplace=True)

In [ ]:
continuous = ['LoanAmount','Loan_Amount_Term','Credit_History','CoapplicantIncome']

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

for var in continuous:
    traindata[var] = traindata[var].astype('float64')
    traindata[var] = scaler.fit_transform(traindata[var].values.reshape(-1, 1))

traindata['ApplicantIncome'] = scaler.fit_transform(traindata['ApplicantIncome'].values.reshape(-1, 1))

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler=MinMaxScaler()

x_scaled = scaler.fit_transform(traindata)
traindata= pd.DataFrame(x_scaled)

In [ ]:
from sklearn.model_selection import train_test_split
X = traindata.iloc[:,:-1].values
Y = traindata.iloc[:,-1].values
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

KerasClassifier wrapper can take arguments, We can use these arguments to further customize the construction of the model.

Importing Gridsearch for hyperparameter tuning to get best parameters for our model. 

using gridsearch in combination of activation function,optimizer.

---

dropout rate, learning rate to find the best combination

In [ ]:
def create_model( act='linear', opt='Adam', dr=0.0,learn_rate=0.01):
      
    model = tf.keras.models.Sequential()
    
    model.add(tf.keras.layers.Dense(units=12, activation=act))
    model.add(tf.keras.layers.Dense(6, activation=act))
    model.add(tf.keras.layers.Dropout(dr))
    # create output layer
    model.add(tf.keras.layers.Dense(1, activation='sigmoid')) 
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    
    return model


model = KerasClassifier(build_fn=create_model, epochs=50, batch_size=32, verbose=0)

# define the grid search parameters
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Nadam']
drops = [0.0, 0.01, 0.1]
learn_rate = [0.001, 0.01, 0.1]
activation = ['softmax', 'softplus', 'relu', 'tanh']

param_grid = dict(learn_rate=learn_rate,opt=optimizer,dr=drops,act=activation)

# search the grid
grid = GridSearchCV(estimator=model, param_grid=param_grid, verbose=2)
grid_result = grid.fit(X_train, Y_train)



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  from ipykernel import kernelapp as app


Fitting 5 folds for each of 216 candidates, totalling 1080 fits
[CV] END .....act=softmax, dr=0.0, learn_rate=0.001, opt=SGD; total time=   3.1s
[CV] END .....act=softmax, dr=0.0, learn_rate=0.001, opt=SGD; total time=   2.0s
[CV] END .....act=softmax, dr=0.0, learn_rate=0.001, opt=SGD; total time=   1.6s
[CV] END .....act=softmax, dr=0.0, learn_rate=0.001, opt=SGD; total time=   2.1s
[CV] END .....act=softmax, dr=0.0, learn_rate=0.001, opt=SGD; total time=   2.0s
[CV] END .act=softmax, dr=0.0, learn_rate=0.001, opt=RMSprop; total time=   1.8s
[CV] END .act=softmax, dr=0.0, learn_rate=0.001, opt=RMSprop; total time=   1.8s
[CV] END .act=softmax, dr=0.0, learn_rate=0.001, opt=RMSprop; total time=   2.2s
[CV] END .act=softmax, dr=0.0, learn_rate=0.001, opt=RMSprop; total time=   1.9s
[CV] END .act=softmax, dr=0.0, learn_rate=0.001, opt=RMSprop; total time=   1.8s
[CV] END .act=softmax, dr=0.0, learn_rate=0.001, opt=Adagrad; total time=   1.5s
[CV] END .act=softmax, dr=0.0, learn_rate=0.0

In [ ]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))


Best: 0.806452 using {'act': 'relu', 'dr': 0.1, 'learn_rate': 0.01, 'opt': 'RMSprop'}
0.676129 (0.020390) with: {'act': 'softmax', 'dr': 0.0, 'learn_rate': 0.001, 'opt': 'SGD'}
0.676129 (0.020390) with: {'act': 'softmax', 'dr': 0.0, 'learn_rate': 0.001, 'opt': 'RMSprop'}
0.594496 (0.150025) with: {'act': 'softmax', 'dr': 0.0, 'learn_rate': 0.001, 'opt': 'Adagrad'}
0.597341 (0.148195) with: {'act': 'softmax', 'dr': 0.0, 'learn_rate': 0.001, 'opt': 'Adadelta'}
0.676129 (0.020390) with: {'act': 'softmax', 'dr': 0.0, 'learn_rate': 0.001, 'opt': 'Adam'}
0.676129 (0.020390) with: {'act': 'softmax', 'dr': 0.0, 'learn_rate': 0.001, 'opt': 'Nadam'}
0.676129 (0.020390) with: {'act': 'softmax', 'dr': 0.0, 'learn_rate': 0.01, 'opt': 'SGD'}
0.676129 (0.020390) with: {'act': 'softmax', 'dr': 0.0, 'learn_rate': 0.01, 'opt': 'RMSprop'}
0.545516 (0.171363) with: {'act': 'softmax', 'dr': 0.0, 'learn_rate': 0.01, 'opt': 'Adagrad'}
0.529190 (0.174886) with: {'act': 'softmax', 'dr': 0.0, 'learn_rate': 0.01

In [ ]:
best_params=grid_result.best_params_
accuracy=grid_result.best_score_

print(best_params,accuracy)

{'act': 'relu', 'dr': 0.1, 'learn_rate': 0.01, 'opt': 'RMSprop'} 0.8064522862434387


# We can conclude that as per given hyper parameters activation function relu, dropout rate 0.1 , learning rate 0.01 , optimizer RMSprop are giving an accuracy of 80.6%
